In [11]:
# import necessary modules
import pybullet as pb
import pybullet_data

import numpy as np
import math

import matplotlib.pyplot as plt

In [12]:
#################################################################
# Forward and Inverse kinematics modules for the serial-2R robot
#################################################################

def forward_kinematics(theta1, theta2, l1, l2):
    '''
    Forward kinematics module for a serial-2R chain.
    The base of the manipulator is assumed to be placed at the
    coordinates [0,0].
    All the joints allow rotation about the positive Z-axis.
    Args:
    --- theta1: Angle between the link l1 and the positive x-axis (in radians)
    --- theta2: Relative angle between link l1 and link l2 (in radians)
    --- l1: Length of link l1 (in m)
    --- l2: Length of link l2 (in m)
    Ret:
    --- [x, y]: Position co-ordinates of the end-effector (in m)
    '''
    x = l1*math.cos(theta1) + l2*math.cos(theta1 + theta2)
    y = l1*math.sin(theta1) + l2*math.sin(theta1 + theta2)
    return [x, y]

def inverse_kinematics(x, y, l1, l2, branch=1):
    '''
    Inverse kinematics modules for the serial-2R manipulator.
    The base of the manipulator is placed at [0,0].
    Axis of rotation is the Z+ axis.
    Args:
    --- x : X co-ordinate of the end-effector
    --- y : Y co-ordinate of the end-effector
    --- l1: Length of link l1
    --- l2: Length of link l2
    --- branch: Branch of the inverse kinematics solution.
    Ret:
    --- valid: Binary variable indicating if the solution is valid or not
    --- [theta1, theta2]: Angles made by link l1 w.r.t X+ axis and the relative
                    angle between links l1 and l2 respectively.
    '''
    a = 2*x*l2
    b = 2*y*l2
    c =  l1*l1 - x*x - y*y  - l2*l2 
    psi = math.atan2(b, a)
    d = -c/math.sqrt(a*a + b*b)
    
    if (d < -1) or (d > 1):
        print("Position out of workspace.")
        return False, [0,0]
    if branch == 1:
        theta12 = psi + math.acos(-c/math.sqrt(a*a + b*b))
    else:
        theta12 = psi - math.acos(-c/math.sqrt(a*a + b*b))
        
    theta1 = math.atan2((y - l2*math.sin(theta12))/l1, (x - l2*math.cos(theta12))/l1)
    return True, [theta1, theta12-theta1]

In [13]:
# Create an instance of the Physics Server and connect to it
client = pb.connect(pb.GUI)
#client = pb.connect(pb.DIRECT)

In [14]:
pb.resetSimulation()

In [15]:
pb.setAdditionalSearchPath(pybullet_data.getDataPath())
#load plane in gui 
plane = pb.loadURDF('plane.urdf')
scara = pb.loadURDF("scara_robot_left.urdf")

In [42]:
plane

0

In [16]:
# set necessary parameters
pb.setGravity(0,0,-9.81)
pb.setTimeStep(0.0001)

In [10]:
position, orientation = pb.getBasePositionAndOrientation(scara)

In [11]:
orientation

(0.0, 0.0, 0.0, 1.0)

In [12]:
#We can ask for the number of joints of the robot.
pb.getNumJoints(scara)

8

In [13]:
# get info of joints
for i in range(8):
    info = pb.getJointInfo(scara, i)
    print(info[1], info[2]) # index 2 has joint type
    #JOINT_REVOLUTE, JOINT_PRISMATIC, JOINT_SPHERICAL, JOINT_PLANAR,JOINT_FIXED = = [0,1,2,3,4]

b'joint_fix' 4
b'rotation1' 0
b'rotation2' 0
b'gripper_joint' 1
b'finger1_joint' 1
b'finger2_joint' 1
b'finger3_joint' 1
b'finger4_joint' 1


In [14]:
#We could as well request the current state of each joint, for example, the positions.

state = pb.getJointStates(scara, range(8))
for i in range(8):
    print(state[i][0])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [15]:
pb.getJointState(scara,1)

(0.0, 0.0, (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0.0)

In [21]:
# enable the motor on joints
pb.setJointMotorControl2(scara, 1, pb.VELOCITY_CONTROL, force=0)
pb.setJointMotorControl2(scara, 2, pb.VELOCITY_CONTROL, force=0)

In [68]:
dt     = 0.001 # Simulation time-step
f      = 0.25 # Frequency of oscillation (1 Hz)
omega  = math.pi*f # Angular frequency
theta0 = 0 # Start position
p_des = np.zeros(100)
for i in range(100):
    t = i*dt
    p_des[i] = np.sin(theta0 + omega*t)


    

p_gain = 1000 # Proportional gain
d_gain = 500 # Derivative gain

error = 0
error_old = 0

pos1 = []
cf = []

# Run the control loop
for i in range(100):
    
    # Get the joint state
    p_act, _, _, _ = pb.getJointState(scara, 2)
    
    # Calculate the control input
    error_old = error
    error = p_des[i] - p_act
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d # PD control
    control_force = np.clip(control_force, -50, 50) # Saturation; to model the torque limit of the motors
    
    # Run the simulation for one time-step
    pb.setJointMotorControl2(scara, 2, pb.TORQUE_CONTROL, force=control_force)
    pb.stepSimulation()
    
    # Store the data for plotting
    pos1.append(p_act)
    cf.append(control_force)

In [31]:
pb.resetSimulation()

In [32]:
# Kinematics for serial-2R
p1 = np.array([1.0, 0.5])
p2 = np.array([0.5, 1.0])
pt_des = p2 # or p2

valid, [theta1, theta2] = inverse_kinematics(pt_des[0], pt_des[1], 1, 1)


dt     = 0.0001 # simulation time-step
p_gain = 200 # Proportional gain
d_gain = 50 # Derivative gain
error  = 0
error_old = 0
desired_pos = np.array([theta1, theta2])
for _ in range(1000):
    pos1, _, _, _ = pb.getJointState(scara,1)
    pos2, _, _, _ = pb.getJointState(scara,2)
    pos = np.array([pos1, pos2])
    error_old = error
    error = desired_pos - pos
    error_d = (error - error_old)/dt
    control_force = p_gain * error + d_gain * error_d
    pb.setJointMotorControlArray(scara, [1,2], pb.TORQUE_CONTROL, forces=control_force)
    pb.stepSimulation()

    
# Check if the robot has reached the desired position
pos1, _, _, _ = pb.getJointState(scara, 1)
pos2, _, _, _ = pb.getJointState(scara, 2)
pt_act = forward_kinematics(pos1, pos2, 1, 1)

print(pt_des)
print(pt_act)

[0.5 1. ]
[0.4999449172336212, 1.0000245321397565]


In [33]:
pb.disconnect()